# 文字預測股票漲跌專案

***根據網路上各類新聞文章預測某一公司股價的漲跌***


台大經研所 羅偉駿

![LinkedIn](https://img.shields.io/badge/linkedin-%230077B5.svg?style=for-the-badge&logo=linkedin&logoColor=white)
![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white)
![Medium](https://img.shields.io/badge/Medium-12100E?style=for-the-badge&logo=medium&logoColor=white)

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import datetime
import os
from args import rawdata_path, workdata_path
import warnings
warnings.filterwarnings('ignore')

# 1. Extract target stock from Dataset

In [2]:
from args import stock_code
from etl_func.etl_data import extract_stock_df

os.chdir(rawdata_path)
stock_df = extract_stock_df(stock_code)

os.chdir(workdata_path)
stock_df.to_parquet(stock_code + '_stock_df.parquet', index=False)
stock_df.head()

,Code,Date,Price
0,2330 台積電,2019-01-02,202.703201
1,2330 台積電,2019-01-03,199.009293
2,2330 台積電,2019-01-04,192.083206
3,2330 台積電,2019-01-07,196.700607
4,2330 台積電,2019-01-08,194.853607


# 2. Cut Article Words

In [4]:
from etl_func.cut_text import Cut_Machine
from args import (
    company, kw_list, data_time, datafile_name,
    kw_title_num, kw_content_num
    )

os.chdir(rawdata_path)
final_df = pd.DataFrame()
for source in datafile_name:
    cut_machine = Cut_Machine(
        articles_source=datafile_name[source],
        data_time=data_time
        )
    cut_machine.filter_article(
        keywords=kw_list,
        title_times=kw_title_num,
        content_times=kw_content_num
        )
    word_df = cut_machine.Pool_sep_all_articles()
    final_df = pd.concat([final_df, word_df], ignore_index=True)
    print('Finish', source, 'with', word_df.shape[0], 'words.')

os.chdir(workdata_path)
final_df.to_parquet(company + '_word_df.parquet', index=False)

Finish bbs with 5482 words.
Finish news2019 with 4621 words.
Finish news2020 with 3307 words.
Finish news2021 with 4214 words.
Finish forum2019 with 2778 words.
Finish forum2020 with 4438 words.
Finish forum2021 with 6423 words.


# 3. Hyperparameters Choosing

In [2]:
from df_func.train import create_train_function, plot_arg_train
import numpy as np
from args import args_class
os.chdir(workdata_path)

In [ ]:
from tqdm import tqdm
from df_func.make_XY import Words_Matrix, feature_X_byChi2
from etl_func.etl_data import transform_stock_df
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from typing import Callable
train_function = Callable[[list], dict]

from args import args_dict, args_class
args = args_class(args_dict)
words_matrix = Words_Matrix(
    word_df=args.word_df,
    stock_df=transform_stock_df(
        args.stock_df, D=args.day_arg, cutoff=args.cutoff_arg
        ),
    data_time=args.data_time,
    stop_words=args.stop_words
    )
X, Y = words_matrix.X_matrix, words_matrix.Y_matrix
print(X.shape, Y.shape)
X = X[ feature_X_byChi2(X, Y, k=args.features_num) ]

In [ ]:
cut_list = [0, 0.01, 0.02, 0.03, 0.04, 0.05]
train_k_feature = create_train_function('cutoff_arg')
results = train_k_feature(cut_list)
np.save('cutoff_arg.npy', pd.DataFrame(results).reset_index())
plot_arg_train('cutoff_arg.npy')

In [ ]:
lag_list = [1, 2, 3, 4, 5]
train_k_feature = create_train_function('day_arg')
results = train_k_feature(lag_list)
np.save('day_arg.csv', pd.DataFrame(results).reset_index())
plot_arg_train('day_arg.csv')

In [ ]:
k_list = [500, 1000, 1500, 2000, 2500]
train_k_feature = create_train_function('features_num')
results = train_k_feature(k_list)
np.save('features_num.csv', pd.DataFrame(results).reset_index())
plot_arg_train('features_num.csv')

# 4. Train & Test 1: 過去測試未來

In [7]:
from sklearn.linear_model import RidgeClassifier
from etl_func.etl_data import transform_stock_df
from df_func.make_XY import Words_Matrix
from df_func.predict import Predict_Machine
from args import (
    word_df, stock_df, stop_words,
    day_arg, cutoff_arg, features_num
    )

train_words_matrix = Words_Matrix(
    word_df=word_df,
    stock_df=transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg),
    data_time=(datetime.date(2019,1,1), datetime.date(2021,5,31)),
    stop_words=stop_words
    )

test_words_matrix = Words_Matrix(
    word_df=word_df,
    stock_df=transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg),
    data_time=(datetime.date(2021,6,1), datetime.date(2021,12,31)),
    stop_words=stop_words
    )

RDclf = RidgeClassifier(alpha = 0.05)

ridge_machine = Predict_Machine(
    train_words_matrix=train_words_matrix,
    test_words_matrix=test_words_matrix,
    features_num=features_num,
    classifier=RDclf
    )

In [19]:
print('Confusion Matrix:\n', ridge_machine.show_confusion())
print()
print('Accuracy Score:', round(ridge_machine.show_accuracy(), 3))

Confusion Matrix:
           True  False
Positive   103      1
Negative    14      0

Accuracy Score: 0.873


# 5. Train & Test 2: 移動回測

In [47]:
from sklearn.linear_model import RidgeClassifier
from etl_func.etl_data import transform_stock_df
from df_func.make_XY import Words_Matrix
from df_func.predict import Predict_Machine, Date_Machine
from args import (
    word_df, stock_df, stop_words,
    day_arg, cutoff_arg, features_num,
    data_time
    )

result_matrix = pd.DataFrame(
            [[0, 0], [0, 0]],
            index=['Positive', 'Negative'],
            columns=['True', 'False'],
            )
date_machine = Date_Machine(train_duration=5, test_duration=2, data_time=data_time)

for i in range(30):
    date_machine.index = i

    train_words_matrix = Words_Matrix(
        word_df=word_df,
        stock_df=transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg),
        data_time=date_machine.train_date,
        stop_words=stop_words
        )

    test_words_matrix = Words_Matrix(
        word_df=word_df,
        stock_df=transform_stock_df(stock_df, D=day_arg, cutoff=cutoff_arg),
        data_time=date_machine.test_date,
        stop_words=stop_words
        )

    RDclf = RidgeClassifier(alpha = 0.05)

    ridge_machine = Predict_Machine(
        train_words_matrix=train_words_matrix,
        test_words_matrix=test_words_matrix,
        features_num=features_num,
        classifier=RDclf
        )
    print(f'{i}th Accuracy Score:', round(ridge_machine.show_accuracy(), 3))

    try:
        result_matrix += ridge_machine.show_confusion()
    except Exception as e:
        print(e)

print('Confusion Matrix in total:\n', result_matrix)

  0%|          | 0/30 [00:00<?, ?it/s]

0th Accuracy Score: 0.941


  3%|▎         | 1/30 [00:11<05:38, 11.68s/it]

1th Accuracy Score: 0.971


  7%|▋         | 2/30 [00:20<04:47, 10.27s/it]

...

28th Accuracy Score: 0.909


 97%|█████████▋| 29/30 [06:10<00:17, 17.24s/it]

29th Accuracy Score: 0.943


100%|██████████| 30/30 [06:26<00:00, 12.87s/it]

Confusion Matrix in total:
           True  False
Positive   756      0
Negative   128      0


# 測試: why knn is nan?

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from df_func.train import train
from args import args_dict, args_class
from df_func.make_XY import Words_Matrix, feature_X_byChi2
from etl_func.etl_data import transform_stock_df
arg_name = 'cutoff_arg'
all_results = {}
for value_ in [0]:
    args_dict[arg_name] = value_
    args = args_class(args_dict)
    words_matrix = Words_Matrix(
        word_df=args.word_df,
        stock_df=transform_stock_df(
            args.stock_df, D=args.day_arg, cutoff=args.cutoff_arg
            ),
        data_time=args.data_time,
        stop_words=args.stop_words
        )
    X, Y = words_matrix.X_matrix, words_matrix.Y_matrix
    X = X[ feature_X_byChi2(X, Y, k=args.features_num) ]

    print("資料中漲跌的比例: \n", Y.value_counts())

    classifier_dict_={ 'kNN': KNeighborsClassifier }
    for classifier in classifier_dict_:
        clf = classifier_dict_[classifier]()
        scores = cross_val_score(estimator=clf, X=X , y=Y, cv=5)
        print(scores)